## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Grindavik,IS,63.8424,-22.4338,50.47,71,40,13.80,scattered clouds
1,1,Lompoc,US,34.6391,-120.4579,69.69,61,0,17.00,clear sky
2,2,Amuntai,ID,-2.4177,115.2494,74.41,98,100,0.74,overcast clouds
3,3,Sao Filipe,CV,14.8961,-24.4956,72.95,70,0,11.72,clear sky
4,4,Gazli,UZ,40.1333,63.4500,76.87,25,45,14.58,scattered clouds


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like at your destination?"))
max_temp = float(input("What is the maximum temperature you would like at your destination?"))

What is the minimum temperature you would like at your destination?65
What is the maximum temperature you would like at your destination?80


In [13]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
chosen_city_data = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
chosen_city_data

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Lompoc,US,34.6391,-120.4579,69.69,61,0,17.00,clear sky
2,2,Amuntai,ID,-2.4177,115.2494,74.41,98,100,0.74,overcast clouds
3,3,Sao Filipe,CV,14.8961,-24.4956,72.95,70,0,11.72,clear sky
4,4,Gazli,UZ,40.1333,63.4500,76.87,25,45,14.58,scattered clouds
5,5,Lahij,YE,13.1667,44.5833,79.52,56,0,7.27,clear sky
...,...,...,...,...,...,...,...,...,...,...
670,670,Marzuq,YE,14.4000,46.4667,72.88,56,14,4.61,few clouds
672,672,Tarakan,ID,3.3000,117.6333,79.57,79,99,0.67,overcast clouds
673,673,Las Margaritas,MX,16.3167,-91.9833,74.97,55,96,6.35,overcast clouds
675,675,Impfondo,CG,1.6381,18.0667,71.73,96,99,3.18,light rain


In [14]:
# 4a. Determine if there are any empty rows.
chosen_city_data.count()

City_ID                202
City                   202
Country                199
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Current Description    202
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = chosen_city_data.dropna()
clean_df.count()

City_ID                199
City                   199
Country                199
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Current Description    199
dtype: int64

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lompoc,US,69.69,clear sky,34.6391,-120.4579,
2,Amuntai,ID,74.41,overcast clouds,-2.4177,115.2494,
3,Sao Filipe,CV,72.95,clear sky,14.8961,-24.4956,
4,Gazli,UZ,76.87,scattered clouds,40.1333,63.4500,
5,Lahij,YE,79.52,clear sky,13.1667,44.5833,
9,Namibe,AO,72.61,few clouds,-15.1961,12.1522,
10,Padang,ID,77.90,moderate rain,-0.9492,100.3543,
11,Lashio,MM,69.30,clear sky,22.9333,97.7500,
13,Rikitea,PF,76.19,light rain,-23.1203,-134.9692,
14,Dingle,PH,76.68,overcast clouds,10.9995,122.6711,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lompoc,US,69.69,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
2,Amuntai,ID,74.41,overcast clouds,-2.4177,115.2494,Hotel Balqis
3,Sao Filipe,CV,72.95,clear sky,14.8961,-24.4956,Hotel Xaguate
4,Gazli,UZ,76.87,scattered clouds,40.1333,63.4500,
5,Lahij,YE,79.52,clear sky,13.1667,44.5833,
9,Namibe,AO,72.61,few clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
10,Padang,ID,77.90,moderate rain,-0.9492,100.3543,Mercure Padang
11,Lashio,MM,69.30,clear sky,22.9333,97.7500,Hotel CF Lashio - Burmese Only
13,Rikitea,PF,76.19,light rain,-23.1203,-134.9692,People ThankYou
14,Dingle,PH,76.68,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY


In [26]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace = True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lompoc,US,69.69,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
2,Amuntai,ID,74.41,overcast clouds,-2.4177,115.2494,Hotel Balqis
3,Sao Filipe,CV,72.95,clear sky,14.8961,-24.4956,Hotel Xaguate
4,Gazli,UZ,76.87,scattered clouds,40.1333,63.4500,NaN
5,Lahij,YE,79.52,clear sky,13.1667,44.5833,NaN
...,...,...,...,...,...,...,...
670,Marzuq,YE,72.88,few clouds,14.4000,46.4667,NaN
672,Tarakan,ID,79.57,overcast clouds,3.3000,117.6333,RedDoorz Syariah near Taman Berlabuh Tarakan
673,Las Margaritas,MX,74.97,overcast clouds,16.3167,-91.9833,Fam Esponda Arguello
675,Impfondo,CG,71.73,light rain,1.6381,18.0667,Auberge La Place de Jules


In [29]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Lompoc,US,69.69,clear sky,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
2,Amuntai,ID,74.41,overcast clouds,-2.4177,115.2494,Hotel Balqis
3,Sao Filipe,CV,72.95,clear sky,14.8961,-24.4956,Hotel Xaguate
9,Namibe,AO,72.61,few clouds,-15.1961,12.1522,Hotel Chik Chik Namibe
10,Padang,ID,77.90,moderate rain,-0.9492,100.3543,Mercure Padang
11,Lashio,MM,69.30,clear sky,22.9333,97.7500,Hotel CF Lashio - Burmese Only
13,Rikitea,PF,76.19,light rain,-23.1203,-134.9692,People ThankYou
14,Dingle,PH,76.68,overcast clouds,10.9995,122.6711,SEGAYA PROPERTY
19,Sorong,ID,77.59,overcast clouds,-0.8833,131.2500,Waigo Splash Hotel
24,Port Alfred,ZA,66.11,clear sky,-33.5906,26.8910,The Halyards Hotel


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))